In [ ]:
import os
import base64
import json
from dotenv import load_dotenv
import openai

In [ ]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
def evaluate_text_with_openai(system_prompt: str, user_prompt: str, model: str='gpt-4o') -> str:
    """Evaluates text using OpenAI ChatCompletion."""
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]
    response = openai.ChatCompletion.create(model=model, messages=messages)
    return response.choices[0].message.content

In [ ]:
def evaluate_images_with_openai(system_prompt: str, image_paths: list, user_prompt: str='', model: str='gpt-4o') -> str:
    """Evaluates images using OpenAI ChatCompletion with base64-encoded image URLs."""
    messages = [{'role': 'system', 'content': system_prompt}]
    for path in image_paths:
        with open(path, 'rb') as f:
            img_b64 = base64.b64encode(f.read()).decode('utf-8')
        # Embed as base64 image_url
        messages.append({'role': 'user', 'image_url': f"data:image/png;base64,{img_b64}"})
    if user_prompt:
        messages.append({'role': 'user', 'content': user_prompt})
    response = openai.ChatCompletion.create(model=model, messages=messages)
    return response.choices[0].message.content

In [ ]:
def run_autograder_full(submission_text: str, image_paths: list):
    """Runs full autograder on text and images."""
    system_prompt = """You are an autograder. Evaluate the following submission based on the provided rubric. Provide structured feedback."""
    # Evaluate text
    text_feedback = evaluate_text_with_openai(system_prompt, submission_text)
    # Evaluate images
    image_feedback = evaluate_images_with_openai(system_prompt, image_paths, 'Evaluate the images based on the same rubric.')
    return {'text_feedback': text_feedback, 'image_feedback': image_feedback}